In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train = pd.read_parquet("C:\\Users\\Thobs\\Desktop\\Portfolio\\Projects\\Data Science Projects\\Machine Learning Project 12 - Bangalore House Prediction\\dataset\\feature_engineered_data\\X_train_df.parquet")
X_test = pd.read_parquet("C:\\Users\\Thobs\\Desktop\\Portfolio\\Projects\\Data Science Projects\\Machine Learning Project 12 - Bangalore House Prediction\\dataset\\feature_engineered_data\\X_test_df.parquet")
y_train = pd.read_parquet("C:\\Users\\Thobs\\Desktop\\Portfolio\\Projects\\Data Science Projects\\Machine Learning Project 12 - Bangalore House Prediction\\dataset\\feature_engineered_data\\y_train.parquet")
y_test = pd.read_parquet("C:\\Users\\Thobs\\Desktop\\Portfolio\\Projects\\Data Science Projects\\Machine Learning Project 12 - Bangalore House Prediction\\dataset\\feature_engineered_data\\y_test.parquet")


In [3]:
X_train.head()

,area_type,availability,location,society,total_sqft,bath,balcony,price,price_per_sqft
4890,8738,17,38,10,1749.0,3.0,1.0,117.00,6689.536878
11222,8738,9860,55,14,1255.0,2.0,2.0,52.35,4171.314741
8696,8738,200,2451,4876,1275.0,2.0,2.0,47.00,3686.274510
1036,2347,9860,60,4876,1280.0,2.0,2.0,58.00,4531.250000
2398,2347,9860,2451,2,1000.0,2.0,2.0,53.00,5300.000000


In [6]:
X_train.shape

(10053, 9)

In [4]:
X_test.head(2)

,area_type,availability,location,society,total_sqft,bath,balcony,price,price_per_sqft
3096,8738,9860,2451,4876,2573.75,4.5,2.0,525.0,4375.000000
7367,2347,9860,12,4876,1030.00,2.0,1.0,60.0,5825.242718


In [5]:
y_test.head(2)

,price
3096,525.0
7367,60.0


In [4]:
y_train.head(3)

,price
4890,117.00
11222,52.35
8696,47.00


In [7]:
# need to drop the price column which is the dependent or target feature
X_train.drop(columns=['price'],inplace=True)
X_test.drop(columns=['price'],inplace=True)

In [8]:
cor_matrix = X_train.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))
print(upper_tri)

                area_type  availability  location   society  total_sqft  \
area_type             NaN      0.135061  0.178788  0.299884    0.152496   
availability          NaN           NaN  0.076524  0.086556    0.051632   
location              NaN           NaN       NaN  0.209021    0.033724   
society               NaN           NaN       NaN       NaN    0.009362   
total_sqft            NaN           NaN       NaN       NaN         NaN   
bath                  NaN           NaN       NaN       NaN         NaN   
balcony               NaN           NaN       NaN       NaN         NaN   
price_per_sqft        NaN           NaN       NaN       NaN         NaN   

                    bath   balcony  price_per_sqft  
area_type       0.216023  0.134102        0.254196  
availability    0.045799  0.114902        0.078345  
location        0.051934  0.008271        0.108480  
society         0.059261  0.067754        0.058942  
total_sqft      0.795126  0.294481        0.406917  
bath  

C:\Users\Cash Crusaders\AppData\Local\Temp\ipykernel_4992\168779758.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k=1).astype(np.bool))


In [9]:
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.80)]
print()
print(to_drop)


[]


In [10]:
# drop the two columns
def drop_correlated(df, to_drop):
    for col in df.columns:
        for i in to_drop:
            if col == i:
                df.drop(col,axis=1,inplace = True)
    df_final = df.copy()
    return df_final

df_X_final = drop_correlated(X_train, to_drop)

In [12]:
df_X_final.head()

,area_type,availability,location,society,total_sqft,bath,balcony,price_per_sqft
4890,8738,17,38,10,1749.0,3.0,1.0,6689.536878
11222,8738,9860,55,14,1255.0,2.0,2.0,4171.314741
8696,8738,200,2451,4876,1275.0,2.0,2.0,3686.274510
1036,2347,9860,60,4876,1280.0,2.0,2.0,4531.250000
2398,2347,9860,2451,2,1000.0,2.0,2.0,5300.000000


In [2]:
import pandas as pd
import numpy as np

import neptune
import neptune.integrations.sklearn as npt_utils

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import r2_score, mean_absolute_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [3]:
df = pd.read_parquet('cleaned_data.parquet')

#seperate target from independent features
X = df.drop(columns=['price'])
y = df['price']

# split train and test dataset
X_train, X_test ,y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

# declare some preprocessing classees
column_transformer = make_column_transformer((OneHotEncoder(sparse=False), [0]), remainder='passthrough')
scaler = StandardScaler()
lin_reg = LinearRegression(normalize=True)

In [ ]:
neptune_run = neptune.init_run(
        project="thobela.sixpence/house-price-pred",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5ZjI2OWZlZi1mZTNkLTQxMDItOGNlYy1iMjdlYzAwNGI1ZWUifQ==")
 